# 门控机制

In [2]:
# 导入必要的库
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
import math

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\Deeplearning\Lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.

首先导入数据集，借助d2l库可以直接导入时光机书数据集，这个库也提供了一些可视化函数；可以通过pip命令下载此库。复制第6章的数据集读取代码也可以达到同样效果（不建议）。

In [ ]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 长短期记忆网络

长短期忆网络的设计灵感来自于计算机的逻辑门。 其中一个门用来从单元中输出条目，称为输出门。另外一个门用来决定何时将数据读入单元，称为输入门。还需要一种机制来重置单元的内容，即遗忘门。<br>首先，让我们自己写一个粗略的长短期记忆网络的前向计算过程，但我们并不会用这个类来训练和预测。

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        # 初始化模型，即各个门的计算参数
        self.W_i = nn.Parameter(torch.randn(input_size, hidden_size))
        self.W_f = nn.Parameter(torch.randn(input_size, hidden_size))
        self.W_o = nn.Parameter(torch.randn(input_size, hidden_size))
        self.W_a = nn.Parameter(torch.randn(input_size, hidden_size))
        self.U_i = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.U_f = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.U_o = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.U_a = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.randn(1, hidden_size))
        self.b_f = nn.Parameter(torch.randn(1, hidden_size))
        self.b_o = nn.Parameter(torch.randn(1, hidden_size))
        self.b_a = nn.Parameter(torch.randn(1, hidden_size))
        self.W_h = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b_h = nn.Parameter(torch.randn(1, hidden_size))
    # 初始化隐藏状态
    def init_state(self, batch_size):
        hidden_state = torch.zeros(batch_size, self.hidden_size)
        cell_state = torch.zeros(batch_size, self.hidden_size)
        return hidden_state, cell_state
    
    def forward(self, inputs, states=None):
        batch_size, seq_len, input_size = inputs.shape
        if states is None:
            states = self.init_state(batch_size)
        hidden_state, cell_state = states
        outputs = []
        for step in range(seq_len):
            inputs_step = inputs[:, step, :]
            i_gate = torch.sigmoid(torch.mm(inputs_step, self.W_i) + torch.mm(hidden_state, self.U_i) + self.b_i)
            f_gate = torch.sigmoid(torch.mm(inputs_step, self.W_f) + torch.mm(hidden_state, self.U_f) + self.b_f)
            o_gate = torch.sigmoid(torch.mm(inputs_step, self.W_o) + torch.mm(hidden_state, self.U_o) + self.b_o)
            c_tilde = torch.tanh(torch.mm(inputs_step, self.W_a) + torch.mm(hidden_state, self.U_a) + self.b_a)
            cell_state = f_gate * cell_state + i_gate * c_tilde
            hidden_state = o_gate * torch.tanh(cell_state)
            y = torch.mm(hidden_state, self.W_h)+ self.b_h
            outputs.append(y)
        return torch.cat(outputs, dim=0), (hidden_state, cell_state)

可以看出，长短时记忆网络模型的架构与基本的循环神经网络单元是相同的，需要对每个时间步分别计算前向过程，但权重更新公式更为复杂。

In [ ]:
class RNNModel(nn.Module):

    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        self.num_directions = 1
        self.linear = nn.Linear(self.num_hiddens, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

# 在第一个时间步，需要初始化一个隐藏状态，由此函数实现
    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

In [ ]:
def train(net, train_iter, vocab, lr, num_epochs, device, use_random_iter=False):

    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
        
    for epoch in range(num_epochs):
        ppl, speed = train_epoch(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            animator.add(epoch + 1, [ppl])
    print('Train Done!')
    torch.save(net.state_dict(),'chapter6.pth')
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')


# 在notebook中，若在训练后直接预测，会直接使用训练后的参数。
# 若想保存模型参数，请解除注释下面的代码，在实例化新的模型进行预测时，请记得读入模型。
# 读入模型的方法请参考前几章实现的Runner类。

#     torch.save(net.state_dict(), 'chapter6.pth')

def train_epoch(net, train_iter, loss, updater, device, use_random_iter):
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
            state.detach_()
        else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
            for s in state:
                s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * d2l.size(y), d2l.size(y))
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

def predict(prefix, num_preds, net, vocab, device):

    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.reshape(torch.tensor(
        [outputs[-1]], device=device), (1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

def grad_clipping(net, theta): 
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')"""
    return torch.device('cpu')

高级API包含了之前介绍的所有配置细节，所以我们可以直接实例化门控循环单元模型。

In [ ]:
vocab_size, num_hiddens, num_epochs, lr= 28, 256, 200, 1
device = try_gpu()
lstm_layer = nn.LSTM(vocab_size, num_hiddens)
model_lstm = RNNModel(lstm_layer, vocab_size)
train(model_lstm, train_iter, vocab, lr, num_epochs, device)

## 门控循环单元（GRU）
门控循环单元是一个稍微简化的变体，通常能够提供同等的效果，并且计算的速度明显更快。<br>我们自己写一个粗略长短期记忆网络的前向计算过程，我们更换了一种写法，用函数来实现这个功能，但我们依旧并不会用这个来训练和预测。

In [ ]:
# 初始化各个“门”的参数
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(inputs, hiddens):
        ctx = device
        param = torch.rand((inputs, hiddens))
        param.to(ctx)
        return param
    
    def three():
        return (normal(num_inputs, num_hiddens),
                normal(num_hiddens, num_hiddens),
                torch.zeros(num_hiddens, device=device))

    W_xz, W_hz, b_z = three()  # 更新门参数
    W_xr, W_hr, b_r = three()  # 重置门参数
    W_xh, W_hh, b_h = three()  # 候选隐状态参数
    # 输出层参数
    W_hq = normal(num_hiddens, num_outputs)
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

# 初始化隐藏状态，作为step=0时的输入
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

# 门控单元
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # @符号为矩阵乘法的运算符号
    for X in inputs:
        Z = torch.sigmoid((X @ W_xz) + (H @ W_hz) + b_z)
        R = torch.sigmoid((X @ W_xr) + (H @ W_hr) + b_r)
        H_tilda = torch.tanh((X @ W_xh) + ((R * H) @ W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = H @ W_hq + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)


In [ ]:
# 此代码使用手动实现的GRU函数
model = d2l.RNNModelScratch(len(vocab), num_hiddens, device, get_params,
                            init_gru_state, gru)
d2l.train_ch8(model, train_iter, vocab, lr, num_epochs, device)

In [ ]:
# 此代码调用Pytorch库的GRU类
gru_layer = nn.GRU(vocab_size, num_hiddens)
model_gru = RNNModel(gru_layer, vocab_size)
train(model_gru, train_iter, vocab, lr, num_epochs, device)

上述代码既完成了包含LSTM层的循环神经网络的训练，也完成了包含GRU的循环神经网络的训练，所以可以分别使用训练过的网络来看看预测结果。‘time’是指定的开始的词语，也可以换成其他英文单词，甚至是乱码。

In [ ]:
predict('time ', 10, model_gru, vocab, device)

In [ ]:
predict('time ', 10, model_lstm, vocab, device)

## 第七章- 优化算法


In [ ]:
# 导入需要的工具包
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from d2l import torch as d2l
from sklearn.datasets import load_iris
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

使用第四章定义的Runner类, 但在本章中不涉及训练中的验证过程.

In [ ]:
class Runner(object):
    def __init__(self, model, optimizer, loss_fn, metric=None, **kwargs):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        # 用于计算评价指标
        self.metric = metric
        
        # 记录训练过程中的评价指标变化
        self.dev_scores = []
        # 记录训练过程中的损失变化
        self.train_step_losses = []
        self.dev_losses = []
        # 记录全局最优评价指标
        self.best_score = 0
   
 
# 模型训练阶段
    def train(self, train_loader, dev_loader=None, **kwargs):
        # 将模型设置为训练模式，此时模型的参数会被更新
        self.model.train()
        
        num_epochs = kwargs.get('num_epochs', 0)
        log_steps = kwargs.get('log_steps', 100)
        save_path = kwargs.get('save_path','final_model.pth')
        eval_steps = kwargs.get('eval_steps', 0)
        # 运行的step数，不等于epoch数
        global_step = 0
        
        if eval_steps:
            if dev_loader is None:
                raise RuntimeError('Error: dev_loader can not be None!')
            if self.metric is None:
                raise RuntimeError('Error: Metric can not be None')
                
        # 遍历训练的轮数
        for epoch in range(num_epochs):
            total_loss = 0
            # 遍历数据集
            for step, data in enumerate(train_loader):
                x, y = data
                logits = self.model(x.float())
                loss = self.loss_fn(logits, y.long())
                total_loss += loss
                if log_steps and global_step%log_steps == 0:
                    print(f'loss:{loss.item():.5f}')
                    
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                global_step += 1
                # 保存当前轮次训练损失的累计值
                self.train_step_losses.append((global_step,loss.item()))
                
            # 每隔一定轮次进行一次验证，由eval_steps参数控制，可以采用不同的验证判断条件
            if eval_steps:
                if (epoch+1)% eval_steps ==  0:

                    dev_score, dev_loss = self.evaluate(dev_loader, global_step=global_step)
                    print(f'[Evalute] dev score:{dev_score:.5f}, dev loss:{dev_loss:.5f}')
                
                    if dev_score > self.best_score:
                        self.save_model(f'model_{epoch+1}.pth')
                    
                        print(f'[Evaluate]best accuracy performance has been updated: {self.best_score:.5f}-->{dev_score:.5f}')
                        self.best_score = dev_score
                    
                # 验证过程结束后，请记住将模型调回训练模式   
                    self.model.train()
            

            
        self.save_model(save_path) 
        print('[Train] Train done')
      
    # 模型评价阶段
    def evaluate(self, dev_loader, **kwargs):
        assert self.metric is not None
        # 将模型设置为验证模式，此模式下，模型的参数不会更新
        self.model.eval()
        global_step = kwargs.get('global_step',-1)
        total_loss = 0
        self.metric.reset()
        
        for batch_id, data in enumerate(dev_loader):
            x, y = data
            logits = self.model(x.float())
            loss = self.loss_fn(logits, y.long()).item()
            total_loss += loss 
            self.metric.update(logits, y)
            
        dev_loss = (total_loss/len(dev_loader))
        self.dev_losses.append((global_step, dev_loss))
        dev_score = self.metric.accumulate()
        self.dev_scores.append(dev_score)
        return dev_score, dev_loss
    
    # 模型预测阶段，
    def predict(self, x, **kwargs):
        self.model.eval()
        logits = self.model(x)
        return logits
    
    # 保存模型的参数
    def save_model(self, save_path):
        torch.save(self.model.state_dict(),save_path)
        
    # 读取模型的参数
    def load_model(self, model_path):
        self.model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

使用第四章定义的可视化函数,观察不同超参数设置下的实验结果

In [ ]:
def plot_training_loss_acc(runner, fig_name, fig_size=(16, 6), sample_step=20, loss_legend_loc='upper right', acc_legend_loc='lower right',
                          train_color = '#8E004D', dev_color = '#E20079', fontsize='x-large', train_linestyle='-', dev_linestyle='--'):
    plt.figure(figsize=fig_size)
    plt.subplot(1,2,1)
    train_items = runner.train_step_losses[::sample_step]
    train_steps = [x[0] for x in train_items]
    train_losses = [x[1] for x in train_items]
    
    plt.plot(train_steps, train_losses, color=train_color, linestyle=train_linestyle, label='Train loss')
    if len(runner.dev_losses) > 0:
        dev_steps = [x[0] for x in runner.dev_losses]
        dev_losses = [x[1] for x in runner.dev_losses]
        plt.plot(dev_steps, dev_losses, color=dev_color, linestyle=dev_linestyle,label='dev loss')
    
    plt.ylabel('loss')
    plt.xlabel('step')
    plt.legend(loc=loss_legend_loc)
    if len(runner.dev_scores) > 0:
        plt.subplot(1,2,2)
        plt.plot(dev_steps, runner.dev_scores, color=dev_color, linestyle=dev_linestyle, label='dev accuracy')
        
        plt.ylabel('score')
        plt.xlabel('step')
        plt.legend(loc=acc_legend_loc)
    # 将绘制结果保存
    plt.savefig(fig_name)
    plt.show()


## 随机梯度下降法

三种随梯度下降法的区别在于批量大小的不同。
第一个实验，我们观察批量大小对梯度下降法的影响。具体来说，我们重新训练第四章的鸢尾花识别任务，使用不同的批量大小，看看会对结果产生什么影响。

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForward,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.act = nn.Sigmoid()
    
    def forward(self, inputs):
        outputs = self.fc1(inputs)
        outputs = self.act(outputs)
        outputs = self.fc2(outputs)
        return outputs

In [ ]:
from torch.utils.data import Dataset, DataLoader

def load_data(shuffle=True):
    x = torch.tensor(load_iris().data)
    y = torch.tensor(load_iris().target)
    
    # 数据归一化
    x_min = torch.min(x, dim=0).values
    x_max = torch.max(x, dim=0).values
    x = (x-x_min)/(x_max-x_min)
    
    if shuffle:
        idx = torch.randperm(x.shape[0])
        x = x[idx]
        y = y[idx]
    return x, y


class IrisDataset(Dataset):
    def __init__(self, mode='train', num_train=120, num_dev=15):
        super(IrisDataset,self).__init__()
        x, y = load_data(shuffle=True)
        if mode == 'train':
            self.x, self.y = x[:num_train], y[:num_train]
        elif mode == 'dev':
            self.x, self.y = x[num_train:num_train + num_dev], y[num_train:num_train + num_dev]
        else:
            self.x, self.y = x[num_train + num_dev:], y[num_train + num_dev:]
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

测试不同批量大小时,需要取消注释相应的行(batch_size).若在notebook中运行时,修改代码后需要重新运行单元格.

In [ ]:
# batch_size = 1
batch_size = 24
# batch_size = 120 

# 分别构建训练集、验证集和测试集
train_dataset = IrisDataset(mode='train')

train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

## 模型训练

请注意: 修改批量大小后,记得修改绘制图像保存的文件名后再运行下面的单元格.\
另一种解决办法是实例化多个Runner类进行不同批量大小的实验,并对不同的Runner实例绘图,但这需要同样实例化多个train_loader.

In [ ]:
input_size = 4
output_size = 3
hidden_size = 6
# 定义模型
model = FeedForward(input_size, hidden_size, output_size)
# 定义损失函数
loss_fn = F.cross_entropy
# 定义优化器
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)
# 实例化辅助runner类
runner = Runner(model, optimizer, loss_fn)
# 模型训练
runner.train(train_loader, num_epochs=50, log_steps=10)
plot_training_loss_acc(runner, 'chapter7_24batchsize.pdf')

## 动量法


In [ ]:
def evaluate_loss(net, data_iter, loss):
    """评估给定数据集上模型的损失

    Defined in :numref:`sec_model_selection`"""
    metric = d2l.Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        X = X.to(torch.float32)
        out = net(X)
#         y = d2l.reshape(y, out.shape)
        l = loss(out, y.long())
        metric.add(d2l.reduce_sum(l), d2l.size(l))
    return metric[0] / metric[1]

def train(trainer_fn, states, hyperparams, data_iter, feature_dim, num_epochs=2):
    """Defined in :numref:`sec_minibatches`"""
    # 初始化模型
    w = torch.normal(mean=0.0, std=0.01, size=(feature_dim, 3),
                     requires_grad=True)
    b = torch.zeros((3), requires_grad=True)
    # 训练模型
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[0, num_epochs], ylim=[0.9, 1.1])
    n, timer = 0, d2l.Timer()
    
    # 这是一个单层线性层     
    net = lambda X: d2l.linreg(X, w, b)
    loss = F.cross_entropy 
    for _ in range(num_epochs):
        for X, y in data_iter:
            X = X.to(torch.float32)
            l = loss(net(X), y.long()).mean()
            l.backward()
            trainer_fn([w, b], states, hyperparams)
            n += X.shape[0]
            if n % 48 == 0:
                timer.stop()
                animator.add(n/X.shape[0]/len(data_iter),
                             (evaluate_loss(net, data_iter, loss),))
                timer.start()
    print(f'loss: {animator.Y[0][-1]:.3f}, {timer.avg():.3f} sec/epoch')

    return timer.cumsum(), animator.Y[0]


接下来，我们实现一个可以满足上面线性层的，带有动量的随机梯度下降法。

In [ ]:
def init_momentum_states(feature_dim):
    v_w = torch.zeros((feature_dim, 3))
    v_b = torch.zeros(3)
    return (v_w, v_b)

def sgd_momentum(params, states, hyperparams):
    for p, v in zip(params, states):
        with torch.no_grad():
            v[:] = hyperparams['momentum'] * v + p.grad
            p[:] -= hyperparams['lr'] * v
            p.grad.data.zero_()

在训练阶段,请将批量大小为24

In [ ]:
lr = 0.02
momentum = 0.9
train(sgd_momentum, init_momentum_states(4),{'lr':lr, 'momentum':momentum}, train_loader, 4)

## 简洁实现

深度学习框架中的优化器早已构建了动量法.\
我们实例化一个sgd类, 并将超参数设置为对应的形式即可.

In [ ]:
input_size = 4
output_size = 3
hidden_size = 6
# 定义模型
model = FeedForward(input_size, hidden_size, output_size)
# 定义损失函数
loss_fn = F.cross_entropy
# 定义优化器, 与之前的实验不同的是,momentum参数被修改
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
# 实例化辅助runner类
runner = Runner(model, optimizer, loss_fn)
# 模型训练
runner.train(train_loader, num_epochs=50, log_steps=10)
plot_training_loss_acc(runner, 'chapter7_24batchsize_momentum.pdf')